In [13]:
#!pip install lazypredict
#!pip install -U sentence-transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from lazypredict.Supervised import LazyClassifier
import json

def write_json(data, path):
    output_dict = {
    'predictions': data.to_dict()
    }
    with open(path, "w", encoding="utf-8") as outfile:
        json.dump(output_dict, outfile, indent=4)


In [4]:
root_drive_dir = '/content/drive/My Drive/Colab Notebooks/DataHub Projects/CrispyWork/edos/data/'

dataset = pd.read_csv(os.path.join(root_drive_dir, "edos_labelled_aggregated.csv"))
dataset = dataset[dataset['label_sexist'] == 'sexist'][["text", "label_category", "label_vector", "split"]]

In [5]:
dataset.shape

(4854, 4)

#### train, dev, test split

In [6]:
train = dataset[dataset['split'] == 'train']
train = train.reset_index(drop=True)
print('Train set shape:', train.shape)

dev = dataset[dataset['split'] == 'dev']
dev = dev.reset_index(drop=True)
print('Validation set shape:', dev.shape)

test = dataset[dataset['split'] == 'test']
test = test.reset_index(drop=True)
print('Test set shape:', test.shape)

Train set shape: (3398, 4)
Validation set shape: (486, 4)
Test set shape: (970, 4)


### load Sentence Transformer

In [7]:
model = SentenceTransformer("all-roberta-large-v1")

# Function to generate sentence embeddings
def generate_embeddings(dataframe):
    embeddings = []
    for sentence in dataframe['text']:
        embeddings.append(model.encode(sentence))
    return embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [8]:
# Generate embeddings for train and test data
train_embeddings = generate_embeddings(train)
dev_embeddings = generate_embeddings(dev)
test_embeddings = generate_embeddings(test)


# Convert the lists of embeddings back to DataFrames
train_features = pd.DataFrame(train_embeddings)
dev_features = pd.DataFrame(dev_embeddings)
test_features = pd.DataFrame(test_embeddings)

train_labels = train['label_vector']
dev_labels = dev['label_vector']
test_labels = test['label_vector']

#### model on dev set

In [9]:
# LazyClassifier
clf = LazyClassifier()
models, dev_predictions = clf.fit(train_features, dev_features, train_labels, dev_labels)

 90%|████████▉ | 26/29 [04:38<00:24,  8.05s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.046038 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 3398, number of used features: 1024
[LightGBM] [Info] Start training from score -4.105591
[LightGBM] [Info] Start training from score -2.593608
[LightGBM] [Info] Start training from score -1.555866
[LightGBM] [Info] Start training from score -1.619197
[LightGBM] [Info] Start training from score -2.832625
[LightGBM] [Info] Start training from score -1.674173
[LightGBM] [Info] Start training from score -2.097856
[LightGBM] [Info] Start training from score -3.972059
[LightGBM] [Info] Start training from score -4.280795
[LightGBM] [Info] Start training from score -3.813454
[LightGBM] [Info] Start training from score -2.577983
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

100%|██████████| 29/29 [07:16<00:00, 15.05s/it]


In [10]:
# save results in json file
json_file_path = "/content/drive/My Drive/Colab Notebooks/DataHub Projects/CrispyWork/edos/02_ST(roberta)_lazyclassifier/TaskC_roberta_lazyclassifier_dev.json"

write_json(dev_predictions, json_file_path)

#### model on test set

In [11]:
# LazyClassifier
clf = LazyClassifier()
models, test_predictions = clf.fit(train_features, test_features, train_labels, test_labels)

 90%|████████▉ | 26/29 [04:29<00:24,  8.15s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044832 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 3398, number of used features: 1024
[LightGBM] [Info] Start training from score -4.105591
[LightGBM] [Info] Start training from score -2.593608
[LightGBM] [Info] Start training from score -1.555866
[LightGBM] [Info] Start training from score -1.619197
[LightGBM] [Info] Start training from score -2.832625
[LightGBM] [Info] Start training from score -1.674173
[LightGBM] [Info] Start training from score -2.097856
[LightGBM] [Info] Start training from score -3.972059
[LightGBM] [Info] Start training from score -4.280795
[LightGBM] [Info] Start training from score -3.813454
[LightGBM] [Info] Start training from score -2.577983
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

100%|██████████| 29/29 [07:05<00:00, 14.69s/it]


In [12]:
json_file_path = "/content/drive/My Drive/Colab Notebooks/DataHub Projects/CrispyWork/edos/02_ST(roberta)_lazyclassifier/TaskC_roberta_lazyclassifier_test.json"

write_json(test_predictions, json_file_path)